In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import xgboost as xgb

from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold, StratifiedKFold

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [8]:
df = pd.read_csv("trainable_df.csv")

In [9]:
df

,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Money_Per_Head,SCALER_Money_Handling_Level,SCALER_Growth,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Health_Score,SCALER_Policy_Start_Date_-_Month,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Health_Conscious_Level,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,SCALER_Number_of_Dependents,SCALER_Occupation_STD_Premium_Amount,SCALER_PC1_Meaningless_df,SCALER_PC2_Meaningless_df,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_ENCODED_Customer_Feedback,SCALER_ENCODED_Occupation_Self-Employed,SCALER_ENCODED_Occupation_Unemployed,SCALER_ENCODED_Marital_Status_Married,SCALER_ENCODED_Marital_Status_Single,Premium Amount
0,-0.389121,-0.974138,-0.064182,-0.255056,-0.066985,-0.494460,-0.323051,28.5,24.535366,1.0,4.862885,-3.0,-0.117678,1.2,0.570401,0.0,0.571429,-0.739699,0.00,0.726836,-0.333333,-0.683947,0.4,0.0,0.222222,-0.956522,-2.155882,-0.5,0.000000,0.467739,0.702766,0.333333,0.7,-0.5,1.0,0.0,1.0,0.0,2869.0
1,0.201496,0.413793,0.218852,0.047050,-0.044286,0.383475,0.471049,-1.0,0.000000,0.0,0.000000,0.0,-0.515532,0.0,0.000000,-1.0,0.000000,0.458999,1.50,-0.273164,-0.333333,-0.683947,0.4,0.0,0.333333,-0.086957,-0.783022,0.5,0.000000,-0.266203,0.201401,0.333333,0.5,0.0,1.0,0.0,0.0,0.0,1483.0
2,0.035581,-0.185345,0.519379,0.868119,-0.134371,0.009355,-0.264725,-1.0,0.000000,0.0,0.000000,0.0,1.273519,0.6,0.000000,0.0,0.000000,-0.557755,0.00,-0.273164,-0.333333,-0.683947,0.4,0.0,0.333333,-0.782609,0.000000,0.5,0.000000,0.933426,0.398433,0.333333,0.6,0.5,1.0,0.0,0.0,0.0,567.0
3,3.210071,-0.995690,6.154735,3.342106,2.640832,1.830617,2.470455,-1.0,0.000000,0.0,0.000000,0.0,-0.777687,0.0,0.575111,-0.5,0.000000,-1.483870,-2.75,0.802161,-0.333333,-0.683947,0.4,0.0,-0.666667,-0.869565,-0.133238,0.0,0.000000,-0.266448,-0.996154,0.666667,0.9,-0.5,1.0,0.0,1.0,0.0,765.0
4,0.419213,0.000000,1.293497,0.552822,1.249683,0.466560,0.304336,0.0,-1.000000,-1.0,-1.000000,1.0,-0.243485,1.2,0.570401,-0.5,0.571429,0.032506,0.00,0.726836,-0.333333,-0.683947,0.4,0.0,-0.333333,-0.869565,0.216978,-0.5,0.000000,-0.998973,-0.195153,-0.333333,-0.1,-0.5,1.0,0.0,0.0,1.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,0.395483,-1.245690,0.182278,0.529106,1.211031,-0.097725,0.285919,-1.0,0.000000,0.0,0.000000,0.0,-0.576158,0.2,0.570401,0.0,0.571429,0.458999,1.50,0.726836,0.666667,0.316053,-0.6,0.0,0.333333,0.391304,-0.783022,-0.5,-0.127755,-0.465794,-0.196157,-0.333333,-0.2,0.0,0.0,0.0,1.0,0.0,NaN
1999996,1.342481,-0.629310,0.717841,0.807278,2.753564,0.923348,1.799409,28.5,24.535366,1.0,4.862885,-3.0,-0.935741,-0.6,-0.429599,1.0,-1.000000,0.458999,1.50,-0.651740,-0.333333,-0.683947,0.4,0.0,0.333333,0.652174,-0.783022,-1.0,0.000000,0.799267,-0.999624,0.333333,0.4,0.5,1.0,0.0,0.0,1.0,NaN
1999997,0.297070,0.715517,0.758972,0.110730,1.050728,0.618831,0.582318,28.5,24.535366,1.0,4.862885,-3.0,-1.021163,0.6,-0.429599,-1.0,-1.000000,0.000000,0.25,-0.651740,0.666667,0.316053,-0.6,0.0,0.555556,-0.652174,0.280920,-1.0,-0.127755,0.933117,0.000567,-1.000000,-1.0,-0.5,0.0,0.0,0.0,1.0,NaN
1999998,0.583327,-0.564655,0.748954,0.301460,0.163031,0.351678,0.915584,28.5,24.535366,1.0,4.862885,-3.0,-0.494730,-0.2,0.000000,0.0,0.000000,0.277595,1.25,-0.273164,-0.333333,-0.683947,0.4,0.0,

#
---
#

# Just Checking!

In [10]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.05:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

In [11]:
give_stats_analysis(df, "Premium Amount")

SCALER_Annual_Income ■■■ Premium Amount-----------✅
SCALER_Credit_Score ■■■ Premium Amount------------✅
SCALER_Determinstic ■■■ Premium Amount------------✅
SCALER_Growth1 ■■■ Premium Amount-----------------✅
SCALER_Money_Per_Head ■■■ Premium Amount----------✅
SCALER_Money_Handling_Level ■■■ Premium Amount----✅
SCALER_Growth ■■■ Premium Amount------------------✅
SCALER_Previous_Claims_MEDIAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims_MEAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims ■■■ Premium Amount---------✅
SCALER_Previous_Claims_STD_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims_MAX_Premium_Amount ■■■ Premium Amount✅
SCALER_Health_Score ■■■ Premium Amount------------✅
SCALER_Policy_Start_Date_-_Month ■■■ Premium Amount✅
SCALER_Number_of_Dependents_MEAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Health_Conscious_Level ■■■ Premium Amount--✅
SCALER_Number_of_Dependents_MEDIAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Insurance_Duration_MEAN_Premiu

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
0,SCALER_Annual_Income,Premium Amount,SpearmanR,-0.060743,0.000000e+00,There is Relationship
1,SCALER_Credit_Score,Premium Amount,SpearmanR,-0.037141,0.000000e+00,There is Relationship
2,SCALER_Determinstic,Premium Amount,SpearmanR,-0.055946,0.000000e+00,There is Relationship
3,SCALER_Growth1,Premium Amount,SpearmanR,-0.052737,0.000000e+00,There is Relationship
4,SCALER_Money_Per_Head,Premium Amount,SpearmanR,-0.052062,0.000000e+00,There is Relationship
5,SCALER_Money_Handling_Level,Premium Amount,SpearmanR,-0.070907,0.000000e+00,There is Relationship
6,SCALER_Growth,Premium Amount,SpearmanR,-0.054155,0.000000e+00,There is Relationship
38,Premium Amount,Premium Amount,SpearmanR,1.000000,0.000000e+00,There is Relationship
7,SCALER_Previous_Claims_MEDIAN_Premium_Amount,Premium Amount,SpearmanR,0.032126,1.918797e-271,There is Relationship
8,SCALER_Previous_Claims_MEAN_Premium_Amount,Premium Amount,SpearmanR,0.031064,6.364897e-254,There is Relationship


#
---
#

# Removing non related columns

In [12]:
# df.drop(columns=["SCALER_ENCODED_Marital_Status_Married", "SCALER_ENCODED_Customer_Feedback", "SCALER_PC2_Meaningless_df", "SCALER_PC1_Meaningless_df", "SCALER_ENCODED_Occupation_Unemployed"], inplace=True)

#
---
#

# Spliting Data

In [13]:
train = df.iloc[:1200000, :]
test = df.iloc[1200000:, :]

train.shape, test.shape

((1200000, 39), (800000, 39))

In [14]:
X = train.drop(columns="Premium Amount")
Y = train["Premium Amount"]
Y_log = np.log(Y)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size=10000)

In [17]:
x_validate.shape

(10000, 38)

In [18]:
test.drop(columns="Premium Amount", inplace=True)

In [19]:
test.shape

(800000, 38)

In [20]:
x_validate.head(3)

,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Money_Per_Head,SCALER_Money_Handling_Level,SCALER_Growth,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Health_Score,SCALER_Policy_Start_Date_-_Month,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Health_Conscious_Level,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,SCALER_Number_of_Dependents,SCALER_Occupation_STD_Premium_Amount,SCALER_PC1_Meaningless_df,SCALER_PC2_Meaningless_df,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_ENCODED_Customer_Feedback,SCALER_ENCODED_Occupation_Self-Employed,SCALER_ENCODED_Occupation_Unemployed,SCALER_ENCODED_Marital_Status_Married,SCALER_ENCODED_Marital_Status_Single
382647,-0.589034,-1.159483,-0.549214,-0.454856,-0.453286,-0.631143,-0.478212,0.0,-1.0,-1.0,-1.0,1.0,0.206352,1.2,0.575111,0.0,0.000000,0.260301,1.0,0.802161,0.0,0.0,0.0,0.0,0.000000,0.956522,0.820980,0.0,0.872245,-0.666052,-0.394736,-0.333333,-0.1,-0.5,0.0,1.0,1.0,0.0
5287,0.399143,0.896552,1.351087,0.532763,1.216991,0.834886,0.288759,0.0,-1.0,-1.0,-1.0,1.0,0.362875,-0.4,0.570401,0.0,0.571429,0.458999,1.5,0.726836,0.0,0.0,0.0,0.0,0.333333,-0.913043,-0.783022,-0.5,0.872245,-0.132199,0.403240,-0.333333,-0.3,0.5,0.0,1.0,0.0,1.0
674505,0.042708,0.508621,-0.168972,0.176529,-0.130502,0.216676,0.012112,0.0,-1.0,-1.0,-1.0,1.0,-0.053252,0.4,0.000000,0.0,0.000000,0.032506,0.0,-0.273164,0.0,0.0,0.0,0.0,-0.333333,0.869565,0.216978,0.5,0.872245,-0.465631,0.304264,0.000000,0.2,0.0,0.0,1.0,0.0,1.0


#
---
#

# Base Model Building

In [21]:
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error

In [22]:
# model = XGBRegressor(verbosity=3)

In [23]:
# model.fit(x_train, np.log(y_train))

In [24]:
# prediction = model.predict(x_validate)

In [25]:
# prediction[prediction < 0] = 0

In [26]:
# root_mean_squared_log_error(y_validate, prediction)

In [27]:
# prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : model.predict(test)})
# prediction

In [28]:
# prediction.to_csv("Dileep's_Submission_Take3.csv", index=False)

#
---
#

# Hyper-tuning -- IT is taking too much time

In [29]:
# from sklearn.model_selection import RandomizedSearchCV

In [30]:
# esimator = XGBRegressor(n_jobs=-1, verbosity=3)

In [31]:
# param_grid = {
#     "n_estimators" : range(50, 1000),
#     "max_depth" : range(5, 30)
# }

In [32]:
# cv_model = RandomizedSearchCV(estimator=esimator, param_distributions=param_grid, verbose=3, n_jobs=-1, cv=2, n_iter=60)

In [33]:
# cv_model.fit(x_train, y_train)

#
---
#

# Let's do Cross-Validation

In [34]:
X["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [35]:
test["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [36]:
lgb_params = {
    'n_estimators': 1175, 
    'learning_rate': 0.02991020706767896, 
    'num_leaves': 79, 
    'max_depth': 13, 
    'min_child_samples': 12, 
    'subsample': 0.9633137940297378, 
    'colsample_bytree': 0.9637121094733179,
    'reg_alpha': 8.846561105667421, 
    'reg_lambda': 3.9007247999299173
}

# cat_params = {
#     'iterations': 3000, 
#     'learning_rate': 0.038365175314273574, 
#     'depth': 11, 
#     'l2_leaf_reg': 3.596285147607088, 
#     'bagging_temperature': 0.2618728648567565
# }

xgb_params = {
    'n_estimators': 1078, 
    'learning_rate': 0.016084079332671603, 
    'max_depth': 10, 
    'min_child_weight': 8, 
    'subsample': 0.8732132237392727, 
    'colsample_bytree': 0.9756972730817159, 
    'reg_alpha': 3.386299962300141, 
    'reg_lambda': 8.964009483088061
}




In [ ]:
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score


splitter = KFold(n_splits=10)
oof = np.zeros(len(X))
models = []


count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # LightGBM model
    lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=1, gpu_device_id=0, verbose=3)

    # CatBoost model
    # cat_model = CatBoostRegressor(**cat_params, verbose=3, task_type= 'GPU')

    # XGBoost model
    xgb_model = XGBRegressor(**xgb_params, verbose=3, tree_method="gpu_hist", predictor= 'gpu_predictor')

    # Voting Regressor
    model = VotingRegressor(
        estimators=[
            ('lgbm', lgbm_model),
            # ('cat', cat_model),
            ('xgb', xgb_model)
        ]
    )

    model.fit(fold_xtrain, fold_ytrain)
    fold_pred = np.maximum(0, model.predict(fold_xtest))
    fold_rmsle = root_mean_squared_log_error(np.expm1(fold_ytest), np.expm1(fold_pred))
    print(f"Fold {count} RMSLE: {fold_rmsle}")
    models.append(model)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2758
[LightGBM] [Info] Number of data points in the train set: 1080000, number of used features: 38
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 34 dense feature groups (37.08 MB) transferred to GPU in 0.112251 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 6.590257
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] 

In [29]:
splitter = KFold(n_splits=10)

In [34]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score
gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=150)
xg_model = XGBRegressor(verbose=3, loss="absolute_error", learning_rate=0.075, n_estimators=100)

count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # print(fold_xtrain.shape, fold_ytrain.shape, fold_xtest.shape, fold_ytest.shape)

    fold_ypred = xg_model.fit(fold_xtrain, fold_ytrain).predict(fold_xtest)
    print(f"The Root mean squared log error for FOLD - {count} is --> {root_mean_squared_log_error(fold_ytest, fold_ypred)}")
    
    folds_df[f"Fold {count}"] = xg_model.predict(test)
    count += 1

      Iter       Train Loss   Remaining Time 
         1           0.7763           26.38m
         2           0.7705           27.03m
         3           0.7659           25.28m
         4           0.7646           23.40m
         5           0.7637           22.34m
         6           0.7628           21.57m
         7           0.7621           20.84m
         8           0.7614           20.30m
         9           0.7613           19.93m
        10           0.7610           19.62m
        11           0.7607           19.29m
        12           0.7605           19.05m
        13           0.7604           18.85m
        14           0.7603           18.62m
        15           0.7603           18.30m
        16           0.7602           18.03m
        17           0.7602           17.76m
        18           0.7600           17.53m
        19           0.7599           17.32m
        20           0.7599           17.14m
        21           0.7598           16.98m
        2

In [41]:
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score

print(f"The Root mean squared log error is --> {np.exp(root_mean_squared_log_error(fold_ytest, fold_ypred))}")
print(f"The R-Square is --> {r2_score(fold_ytest, fold_ypred)}")


The Root mean squared log error is --> 1.179851149390503
The R-Square is --> 0.020856313374837288


In [42]:
folds_df.to_csv("folds_prediction.csv", index=False)

#
---
#

In [43]:
preds = pd.read_csv("folds_prediction.csv")
preds

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10
0,6.930233,6.867230,6.853147,6.847913,6.857263,6.854217,6.913714,6.893065,6.868375,6.894655
1,6.784124,6.767386,6.766795,6.797342,6.829831,6.810085,6.792542,6.824785,6.780036,6.798060
2,6.800640,6.794048,6.812201,6.785757,6.822497,6.782858,6.790993,6.836889,6.798840,6.795229
3,6.788893,6.792624,6.812714,6.814927,6.826816,6.815382,6.807435,6.807562,6.795234,6.824899
4,6.762104,6.759123,6.743320,6.717056,6.756136,6.762728,6.744538,6.753263,6.770463,6.740757
...,...,...,...,...,...,...,...,...,...,...
799995,6.947190,6.921992,6.949191,6.940423,6.967889,6.949643,6.963356,6.941865,6.948702,6.948496
799996,6.959979,6.834935,6.801705,6.830459,6.831910,6.672974,6.764789,6.818086,6.863477,6.845015
799997,6.862624,6.863395,6.896912,6.870820,6.928916,6.846351,6.861084,6.857269,6.891078,6.886685
799998,6.860557,6.850789,6.793036,6.838752,6.842770,6.831941,6.838984,6.856140,6.828237,6.854927


In [44]:
pred = preds.mean(axis=1).apply(np.exp)

In [45]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : pred})
prediction

,id,Premium Amount
0,1200000,970.664834
1,1200001,893.457324
2,1200002,899.640460
3,1200003,905.646235
4,1200004,854.869447
...,...,...
799995,1999995,1040.934863
799996,1999996,918.124306
799997,1999997,969.241251
799998,1999998,934.127768


In [46]:
prediction.to_csv("Dileep's_Submission_Take21.csv", index=False)

In [60]:
# Take 20 did after Feature engineering
# Before -- 821 Rank
# After -- 757

#
---
#

# Lets implement... `STACKING`

In [106]:
X.fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [107]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_log_error

In [111]:
lir_model = LinearRegression()
knn_model = KNeighborsRegressor()
rf_model = RandomForestRegressor(verbose=3, n_estimators=40)
gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=250)
xgb_model = XGBRegressor(verbose=3)
ridge_model = Ridge()
if_model = IsolationForest(verbose=3)

In [112]:
stacked_model = StackingRegressor(estimators=[
                    ("rf_model", rf_model),
                    # ("knn_model", knn_model),
                    ("xgb_model", xgb_model),
                    ("gb_model", gb_model),
                    # ("lir_model", lir_model)
                ], final_estimator=lir_model, verbose=3)

In [113]:
gb_model.fit(X, np.log(Y))

      Iter       Train Loss   Remaining Time 
         1           0.7717           34.10m
         2           0.7642           32.46m
         3           0.7623           30.65m
         4           0.7610           30.38m
         5           0.7604           30.08m
         6           0.7600           29.70m
         7           0.7598           29.28m
         8           0.7594           29.04m
         9           0.7592           28.73m
        10           0.7591           28.47m
        11           0.7591           28.43m
        12           0.7590           28.31m
        13           0.7589           28.50m
        14           0.7589           28.71m
        15           0.7588           28.62m
        16           0.7587           28.36m
        17           0.7587           28.18m
        18           0.7586           28.15m
        19           0.7584           27.90m
        20           0.7582           28.12m
        21           0.7582           28.06m
        2

GradientBoostingRegressor(learning_rate=0.5, loss='absolute_error',
                          n_estimators=250, verbose=3)

In [114]:
prediction = gb_model.predict(x_validate)

In [115]:
# prediction[prediction < 0] = prediction.mean()

In [116]:
root_mean_squared_log_error(y_validate, prediction)

np.float64(4.67300233947815)

In [122]:
test.fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [123]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : np.exp(gb_model.predict(test))})
prediction

,id,Premium Amount
0,1200000,957.075734
1,1200001,920.077180
2,1200002,905.662536
3,1200003,905.194894
4,1200004,833.240238
...,...,...
799995,1999995,1036.266459
799996,1999996,903.047919
799997,1999997,1005.565005
799998,1999998,935.444136


In [124]:
prediction.to_csv("Dileep's_Submission_Take14.csv", index=False)